In [1]:
import os
SPARK_HOME="/opt/spark"
os.environ["PYSPARK_PYTHON"] = "python3"
# os.environ["PYSPARK_PYTHON"]="python"
os.environ["PYSPARK_DRIVER_PYTHON"]="jupyter"
os.environ["PYSPARK_DRIVER_PYTHON_OPTS"]="notebook"
os.environ["SPARK_HOME"] = SPARK_HOME
os.environ["SPARK_CLASSPATH"] = "/opt/spark/jars/sqljdbc4.jar"

 

from pyspark.sql import SparkSession
spark = SparkSession \
        .builder \
        .appName("Task_3") \
        .getOrCreate()

In [2]:
#from pyspark.sql import SQLContext
# sqlContext = SQLContext(spark)
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [5]:
df_path = "/home/rupeshr/Desktop/TSA_Python/sample_email.csv"
df =spark.read.format("com.databricks.spark.csv")\
                    .option("multiline", "true")\
                    .option("header", "true")\
                    .option("inferSchema", "true")\
                    .option("sep", ',')\
                    .option('escape', '\"')\
                    .option("allowSingleQuotes", "true")\
                    .option("ignoreLeadingWhiteSpace", "true")\
                    .option("ignoreTrailingWhiteSpace", "true")\
                    .load(df_path)
df.printSchema()
df.show()

root
 |-- email: string (nullable = true)

+--------------------+
|               email|
+--------------------+
|rupeshrupees@gmai...|
|    kokojambo@mon.eu|
|         lol@fsa.org|
|         ii@koko.com|
|rupesh.subramani@...|
+--------------------+



In [3]:
from pyspark.sql.types import *
from pyspark.ml.feature import *
from pyspark.sql.functions import UserDefinedFunction, udf
from pyspark.sql.functions import lit
import ast
import numpy as np
import traceback
from math import ceil,floor
import plotly.express as px

In [102]:
df_path ="/home/rupeshr/Desktop/TSA_Python/dataset/dataset.csv"
df = spark.read.format("com.databricks.spark.csv").option("header", "true").option("inferSchema", "true").load(df_path)
df.show()

+----------------+--------+---------+-------------------+
|SystemCodeNumber|Capacity|Occupancy|        LastUpdated|
+----------------+--------+---------+-------------------+
|     BHMBCCMKT01|     577|       61|2016-10-04 07:59:42|
|     BHMBCCMKT01|     577|       64|2016-10-04 08:25:42|
|     BHMBCCMKT01|     577|       80|2016-10-04 08:59:42|
|     BHMBCCMKT01|     577|      107|2016-10-04 09:32:46|
|     BHMBCCMKT01|     577|      150|2016-10-04 09:59:48|
|     BHMBCCMKT01|     577|      177|2016-10-04 10:26:49|
|     BHMBCCMKT01|     577|      219|2016-10-04 10:59:48|
|     BHMBCCMKT01|     577|     null|2016-10-04 11:25:47|
|     BHMBCCMKT01|     577|     null|2016-10-04 11:59:44|
|     BHMBCCMKT01|     577|     null|2016-10-04 12:29:45|
|     BHMBCCMKT01|     577|     null|2016-10-04 13:02:48|
|     BHMBCCMKT01|     577|     null|2016-10-04 13:29:45|
|     BHMBCCMKT01|     577|     null|2016-10-04 14:02:47|
|     BHMBCCMKT01|     577|     null|2016-10-04 14:29:49|
|     BHMBCCMK

In [104]:
val = df.groupby().max('Occupancy').collect()[0]
vals = df.groupby().min('Occupany').collect()[0]
maxi = [val[0]]
mini = [vals[0]]
print(maxi, mini)

AnalysisException: Cannot resolve column name "Occupany" among (SystemCodeNumber, Capacity, Occupancy, LastUpdated);

In [85]:
x=[20,30,40,50,60,70]
s= mini + x + maxi
print(s)
s=list(set(s))
s.sort(reverse=False)
print(s)

[18, 20, 30, 40, 50, 60, 70, 70]
[18, 20, 30, 40, 50, 60, 70]


In [28]:
bucketizer = Bucketizer(splits=s, inputCol="Age",outputCol="age_result")

In [29]:
bucketed = bucketizer.transform(df)
bucketed.show()

+----------+------+---+------------------+----------------------+----------+
|CustomerID| Genre|Age|Annual Income (k$)|Spending Score (1-100)|age_result|
+----------+------+---+------------------+----------------------+----------+
|         1|  Male| 19|                15|                    39|       0.0|
|         2|  Male| 21|                15|                    81|       1.0|
|         3|Female| 20|                16|                     6|       1.0|
|         4|Female| 23|                16|                    77|       1.0|
|         5|Female| 31|                17|                    40|       2.0|
|         6|Female| 22|                17|                    76|       1.0|
|         7|Female| 35|                18|                     6|       2.0|
|         8|Female| 23|                18|                    94|       1.0|
|         9|  Male| 64|                19|                     3|       5.0|
|        10|Female| 30|                19|                    72|       2.0|

In [63]:
y = []
for i in range(len(s[:-1])):
    q = (np.round(s[i],6), np.round(s[i+1],6))
    y.append(q)
print(y)

[(18, 20), (20, 30), (30, 40), (40, 50), (50, 60), (60, 70)]


In [45]:
len(s[:-1])

6

In [81]:
def testUDF(col, y):
    if col:
        y = ast.literal_eval(str(y))
        return str(y[int(col)])
    elif '0' in str(col):
        return str(ast.literal_eval(str(y))[0])
    else:
        return None
myudf = udf(testUDF, StringType())
f = bucketed.withColumn('age_source', myudf('age_result', lit(str(y))))
sd = f.drop("age_result")

In [66]:
print(y)

[(18, 20), (20, 30), (30, 40), (40, 50), (50, 60), (60, 70)]


In [80]:
ast.literal_eval(str(y))[0]

(18, 20)

In [71]:
print(y[0])

(18, 20)


In [82]:
sd.show()

+----------+------+---+------------------+----------------------+----------+
|CustomerID| Genre|Age|Annual Income (k$)|Spending Score (1-100)|age_source|
+----------+------+---+------------------+----------------------+----------+
|         1|  Male| 19|                15|                    39|  (18, 20)|
|         2|  Male| 21|                15|                    81|  (20, 30)|
|         3|Female| 20|                16|                     6|  (20, 30)|
|         4|Female| 23|                16|                    77|  (20, 30)|
|         5|Female| 31|                17|                    40|  (30, 40)|
|         6|Female| 22|                17|                    76|  (20, 30)|
|         7|Female| 35|                18|                     6|  (30, 40)|
|         8|Female| 23|                18|                    94|  (20, 30)|
|         9|  Male| 64|                19|                     3|  (60, 70)|
|        10|Female| 30|                19|                    72|  (30, 40)|

In [42]:
sd.show()

+----------+------+---+------------------+----------------------+----------+
|CustomerID| Genre|Age|Annual Income (k$)|Spending Score (1-100)|age_source|
+----------+------+---+------------------+----------------------+----------+
|         1|  Male| 19|                15|                    39|      null|
|         2|  Male| 21|                15|                    81|  (20, 30)|
|         3|Female| 20|                16|                     6|  (20, 30)|
|         4|Female| 23|                16|                    77|  (20, 30)|
|         5|Female| 31|                17|                    40|  (30, 40)|
|         6|Female| 22|                17|                    76|  (20, 30)|
|         7|Female| 35|                18|                     6|  (30, 40)|
|         8|Female| 23|                18|                    94|  (20, 30)|
|         9|  Male| 64|                19|                     3|  (60, 70)|
|        10|Female| 30|                19|                    72|  (30, 40)|

In [105]:
def custom_binning(data, column, values, new_column):
    try:
        if data.count() == 1:
            columnValue = data.select(column).collect()[0][0]
            data = data.withColumn(new_column, lit(str(str(columnValue))))
            return data
        else:
            val = data.groupby().max(column[0]).collect()[0]
            vals = data.groupby().min(column[0]).collect()[0]
            c = [val[0]] # max
            d = [vals[0]] # min
            s= d + values + c
            print(s)
            s=list(set(s))
            s.sort(reverse=False)
            print(s)
            bucketizer = Bucketizer(splits=s, inputCol=column[0],outputCol='temp5_binning')
            bucketed = bucketizer.transform(data)
            y = []
            for i in range(len(s[:-1])):
                q = (np.round(s[i],6), np.round(s[i+1],6))
                y.append(q)
            print(y)
            def testUDF(col, y):
                if col:
                    y = ast.literal_eval(str(y))
                    return str(y[int(col)])
                elif '0' in str(col):
                    return str(ast.literal_eval(str(y))[0])
                else:
                    return None
            myudf = udf(testUDF, StringType())
            x1 = bucketed.withColumn(new_column, myudf("temp5_binning", lit(str(y))))
            x1 = x1.drop("temp5_binning")
            return x1
    except Exception as e:
        traceback.print_exc()
        # print(e)
        return "error"

In [109]:
s=custom_binning(df,['Occupancy'],[200,500,800,1000,1500,2000],'Occ_source')
s.show()

[-8, 200, 500, 800, 1000, 1500, 2000, 4327]
[-8, 200, 500, 800, 1000, 1500, 2000, 4327]
[(-8, 200), (200, 500), (500, 800), (800, 1000), (1000, 1500), (1500, 2000), (2000, 4327)]
+----------------+--------+---------+-------------------+----------+
|SystemCodeNumber|Capacity|Occupancy|        LastUpdated|Occ_source|
+----------------+--------+---------+-------------------+----------+
|     BHMBCCMKT01|     577|       61|2016-10-04 07:59:42| (-8, 200)|
|     BHMBCCMKT01|     577|       64|2016-10-04 08:25:42| (-8, 200)|
|     BHMBCCMKT01|     577|       80|2016-10-04 08:59:42| (-8, 200)|
|     BHMBCCMKT01|     577|      107|2016-10-04 09:32:46| (-8, 200)|
|     BHMBCCMKT01|     577|      150|2016-10-04 09:59:48| (-8, 200)|
|     BHMBCCMKT01|     577|      177|2016-10-04 10:26:49| (-8, 200)|
|     BHMBCCMKT01|     577|      219|2016-10-04 10:59:48|(200, 500)|
|     BHMBCCMKT01|     577|     null|2016-10-04 11:25:47|      null|
|     BHMBCCMKT01|     577|     null|2016-10-04 11:59:44|     

In [100]:
s=['1','2']
print(int(s))

TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'